In [1]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.difficulty = 4  # Initial difficulty
        self.block_time = 10  # Desired time to mine a block (in seconds)
        self.adjustment_interval = 10  # Number of blocks before difficulty adjustment

    def calculate_dynamic_difficulty(self):
        if len(self.chain) < self.adjustment_interval:
            return self.difficulty

        last_blocks = self.chain[-self.adjustment_interval:]
        actual_time = last_blocks[-1].timestamp - last_blocks[0].timestamp

        if actual_time < (self.block_time * self.adjustment_interval) // 2:
            self.difficulty += 1  # Increase difficulty
        elif actual_time > (self.block_time * self.adjustment_interval) * 2:
            self.difficulty -= 1  # Decrease difficulty
        return self.difficulty


In [2]:
class Blockchain:
    def resolve_forks(self, competing_chains):
        # Compare chain lengths
        longest_chain = max(competing_chains, key=len)
        # Alternatively, select the chain with the highest cumulative difficulty
        most_difficult_chain = max(competing_chains, key=self.calculate_total_difficulty)
        return longest_chain if len(longest_chain) > len(most_difficult_chain) else most_difficult_chain

    def calculate_total_difficulty(self, chain):
        return sum(block.difficulty for block in chain)


In [3]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

class Student:
    def __init__(self, student_id):
        self.student_id = student_id
        self.private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
        self.public_key = self.private_key.public_key()

    def get_public_key_pem(self):
        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode()

    def sign_attendance(self, data):
        # Sign attendance with the private key
        from cryptography.hazmat.primitives.asymmetric import padding
        from cryptography.hazmat.primitives import hashes

        signature = self.private_key.sign(
            data.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return signature


In [4]:
class AttendanceRecord:
    def __init__(self, student_id, event_id, timestamp, signature):
        self.student_id = student_id
        self.event_id = event_id
        self.timestamp = timestamp
        self.signature = signature

    def to_dict(self):
        return {
            "student_id": self.student_id,
            "event_id": self.event_id,
            "timestamp": self.timestamp,
            "signature": self.signature.hex()
        }


In [ ]:
class Miner:
    def __init__(self, blockchain):
        self.blockchain = blockchain

    def mine_block(self, attendance_records):
        valid_records = self.validate_records(attendance_records)
        new_block = Block(
            index=len(self.blockchain.chain),
            previous_hash=self.blockchain.chain[-1].hash if self.blockchain.chain else "0",
            records=valid_records,
            difficulty=self.blockchain.difficulty
        )
        self.blockchain.chain.append(new_block)

    def validate_records(self, records):
        # Validate the signature of each record
        from cryptography.hazmat.primitives.asymmetric import padding
        from cryptography.hazmat.primitives import hashes
        valid_records = []
        for record in records:
            try:
                public_key = serialization.load_pem_public_key(record.public_key.encode())
                public_key.verify(
                    bytes.fromhex(record.signature),
                    f"{record.student_id}{record.event_id}{record.timestamp}".encode(),
                    padding.PSS(
                        mgf=padding.MGF1(hashes.SHA256()),
                        salt_length=padding.PSS.MAX_LENGTH
                    ),
                    hashes.SHA256()
                )
                valid_records.append(record)
            except Exception:
                pass  # Invalid record
        return valid_records


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/attendance', methods=['GET'])
def query_attendance():
    student_id = request.args.get('student_id')
    attendance = [block.records for block in blockchain.chain if any(
        record.student_id == student_id for record in block.records
    )]
    return jsonify(attendance)

if __name__ == '__main__':
    app.run(debug=True)


In [5]:
class AttendanceRecord:
    def __init__(self, student_id, event_id, timestamp, class_id=None):
        self.student_id = student_id
        self.event_id = event_id
        self.timestamp = timestamp
        self.class_id = class_id

    def to_dict(self):
        return {
            "student_id": self.student_id,
            "event_id": self.event_id,
            "timestamp": self.timestamp,
            "class_id": self.class_id
        }


In [6]:
class Blockchain:
    def __init__(self):
        self.chain = []

    def get_attendance_by_student(self, student_id, start_date=None, end_date=None):
        records = []
        for block in self.chain:
            for record in block.records:
                if record.student_id == student_id:
                    # Filter by date if specified
                    if start_date and end_date:
                        if start_date <= record.timestamp <= end_date:
                            records.append(record.to_dict())
                    else:
                        records.append(record.to_dict())
        return records


In [ ]:
class Blockchain:
    def get_attendance_by_class(self, class_id, start_date=None, end_date=None):
        records = []
        for block in self.chain:
            for record in block.records:
                if record.class_id == class_id:
                    # Filter by date if specified
                    if start_date and end_date:
                        if start_date <= record.timestamp <= end_date:
                            records.append(record.to_dict())
                    else:
                        records.append(record.to_dict())
        return records


In [ ]:
from flask import Flask, request, jsonify
from datetime import datetime

app = Flask(__name__)

# Mock blockchain instance
blockchain = Blockchain()

@app.route('/query/student', methods=['GET'])
def query_by_student():
    student_id = request.args.get('student_id')
    start_date = request.args.get('start_date')  # Format: YYYY-MM-DD
    end_date = request.args.get('end_date')      # Format: YYYY-MM-DD

    # Convert dates to datetime objects if provided
    if start_date:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if end_date:
        end_date = datetime.strptime(end_date, "%Y-%m-%d")

    records = blockchain.get_attendance_by_student(student_id, start_date, end_date)
    return jsonify(records)

@app.route('/query/class', methods=['GET'])
def query_by_class():
    class_id = request.args.get('class_id')
    start_date = request.args.get('start_date')  # Format: YYYY-MM-DD
    end_date = request.args.get('end_date')      # Format: YYYY-MM-DD

    # Convert dates to datetime objects if provided
    if start_date:
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if end_date:
        end_date = datetime.strptime(end_date, "%Y-%m-%d")

    records = blockchain.get_attendance_by_class(class_id, start_date, end_date)
    return jsonify(records)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
